In [15]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [19]:
from Classes.cognitive_model_agents import MODELS, M1_MODELS, M2_MODELS, M3_MODELS
from Classes.parameter_recovery import ParameterFit
from Config.config import PATHS

## Fit to 2-player data

In [17]:
# Create paths for data and results
data_folder = Path('..', 'data', 'human')
folder_2P = Path('..', 'reports', 'MLE')
folder_2P.mkdir(parents=True, exist_ok=True)

file_name = f'best_fit_2P_with_FRA.json'
best_fit_file_2P = Path(folder_2P, file_name)

In [18]:
#Load 2P data

file_name = '2-player-UR.csv'
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data.head(2)

Loading data from ../data/human/2-player-UR.csv...


,source,treatment,group,round,player,choice,score,attendance,ac_score,threshold,num_players,room,decision
0,human,score + grid,Grupo-0001,1,405186428721292,0,0,1,25,0.5,2,Grupo-0001,0
1,human,score + grid,Grupo-0001,1,728608683559397,1,1,1,25,0.5,2,Grupo-0001,1


In [ ]:
# check_out_these = ['Attendance-M2', 'MFP-M2', 'Payoff-M2']
# check_out_these = ['Fairness-M2']
# my_models = [model for model in MODELS if model.name() in check_out_these]

ParameterFit.run(
    data=data,
    model_list=MODELS,
    # model_list=my_models,
    # model_list=M2_MODELS,
    # model_list=M3_MODELS,
    best_fit_path=best_fit_file_2P,
    new_file=False
)

## Fit to all player data

In [15]:
# Create paths for data and results
folder_all = Path('..', 'reports', 'MLE')
folder_all.mkdir(parents=True, exist_ok=True)

best_fit_file_all = Path(folder_all, f'best_fit_all_with_FRA.json')

In [42]:
# Load data into a dataframe
file_names = [
	# '2-player-UR.csv',
	# '3-player-IU.csv',
	# '4-player-IU.csv',
	# '5-player-IU.csv',
	# '6-player-IU.csv',
	# '7-player-IU.csv',
	# '8-player-IU.csv',
	# '9-player-IU.csv',
	'11-player-IU.csv',
	# '12-player-IU.csv',
]
# file_names = ['multi-player.csv']
data_folder = PATHS['human_data']
df_list = list()
for file_name in file_names:
	file = data_folder / Path(file_name)
	print(f'Loading data from {file}...')
	df = pd.read_csv(file)
	df_list.append(df)
data = pd.concat(df_list, ignore_index=True)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head()

Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/11-player-IU.csv...


,threshold,round,player,decision,score,num_players,group
0,0.272727,1.0,484130782214859,0.0,0.0,11,13
1,0.272727,1.0,324279369615055,0.0,0.0,11,13
2,0.272727,1.0,161168239648852,1.0,-1.0,11,13
3,0.272727,1.0,891802350848838,0.0,0.0,11,13
4,0.272727,1.0,210091536125670,1.0,-1.0,11,13


In [40]:
my_models = [Titan, AttendanceM2]
ParameterFit.run(
    data=data,
    # model_list=MODELS,
    model_list=my_models,
    best_fit_path=best_fit_file_all,
    new_file=False
) 

Fitting models...:   0%|                                  | 0/2 [00:00<?, ?it/s]

Fitting data to model FRA+Payoff+Attendance...
Creating parameter recovery class...
Running bayesian optimizer...
Finding deviance for 11 players and threshold 0.6363636363636364...
|   iter    |  target   | invers... |   bias    | learni... | len_hi... | max_re... |   delta   |
-------------------------------------------------------------------------------------------------
| 1         | -1369.122 | 27.272386 | 0.7203244 | 0.0001143 | 4.0233257 | 2.3208030 | 0.0092338 |
| 2         | -2388.464 | 12.734393 | 0.3455607 | 0.3967674 | 6.3881673 | 4.7727506 | 0.0685219 |
| 3         | -1785.042 | 13.880491 | 0.8781174 | 0.0273875 | 7.7046751 | 4.7557432 | 0.0558689 |
| 4         | -2908.993 | 9.8443771 | 0.1981014 | 0.8007445 | 10.682615 | 3.8208176 | 0.0692322 |
| 5         | -4989.377 | 56.212516 | 0.8946066 | 0.0850442 | 1.3905478 | 2.5284737 | 0.0878142 |
